Лабораторная работа №5

Импорт библиотек

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
from collections import Counter
import csv
import sys
import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from torch.utils.data import TensorDataset, DataLoader
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
sys.path.insert(1, "C:\\Users\\ksush\\OneDrive\\Рабочий стол\\python-v8\\Lab4")
from func import load_reviews

Считываем путь

In [51]:
data_folder = 'C:\\Users\\ksush\\OneDrive\\Рабочий стол\\python-v8\\dataset'
df = load_reviews(os.path.join(data_folder, 'normal_dataset.csv'))

In [52]:
df['Class'] = df['Class'].apply(lambda x: 1 if x == 'good' else 0)

Удаление пустых строк

In [53]:
df.dropna(inplace=True)

Обработка текста отзыва

In [54]:
remove_non_alphabets = lambda x: re.sub(r'[^\w\s]', ' ', x, flags=re.UNICODE)

tokenize = lambda x: word_tokenize(x)

ps = PorterStemmer()
stem = lambda w: [ps.stem(x) for x in w]

lemmatizer = WordNetLemmatizer()
leammtizer = lambda x: [lemmatizer.lemmatize(word) for word in x]

counter = lambda x: Counter(x)
sort_words = lambda x: list((dict(sorted(x.items(), key=lambda item: item[1], reverse=True)[:500])).keys())

In [55]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ksush\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ksush\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ksush\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
print('Start processing')
df['review'] = df['review'].apply(remove_non_alphabets)
print('removed non alphabets')
df['review'] = df['review'].apply(tokenize)
print('tokenized')
df['review'] = df['review'].apply(stem)
print('stemmed')
df['review'] = df['review'].apply(leammtizer)
print('lemmatized')
df['review'] = df['review'].apply(counter)
print('counted')
df['review'] = df['review'].apply(sort_words)
print('sorted')
df['review'] = df['review'].apply(lambda x: ' '.join(x))
print('End of processing')

Start processing


KeyError: 'review'

Start processing


KeyError: 'review'